# Purpose:
- To add DLC eye data to design matrix
- As in bod.eye_tracking
    - Should have at least pupil_area, eye_width, and likely_blink
- See if implementing into COMB is easy (patch)

# Conclusion:
- package issue, involving pytables
    - pd.read_hdf does not work or errors out
- Hold this off for now, continue dev GLM fitting

In [1]:
import h5py
import numpy as np
import pandas as pd
from pathlib import Path

# notebook dev
%load_ext autoreload
%autoreload 2
%matplotlib inline
import capsule_utils
import load_data
import design_matrix_tools as dmtools
import kernel_tools as ktools

In [2]:
session_name = 'multiplane-ophys_721291_2024-05-16_08-57-00'

data_dir = Path('/root/capsule/data/')
raw_path = data_dir / session_name
processed_path = list(data_dir.glob(f'{session_name}_processed*'))[0]

opids = []
for plane_folder in processed_path.glob("*"):
    if plane_folder.is_dir() and not plane_folder.stem.startswith("nextflow"):
        opid = plane_folder.stem
        opids.append(opid)

# bod_list = []
# for opid in opids:
#     bod = load_data.load_plane_data(session_name, opid=opid)
#     bod = capsule_utils.add_trials_to_bod(bod)
#     bod_list.append(bod)
# bod = bod_list[0]
bod = load_data.load_plane_data(session_name, opid=opids[0])

File rig.json not found in /root/capsule/data/multiplane-ophys_721291_2024-05-16_08-57-00


In [3]:
eye_dir = list(data_dir.glob(f'{session_name}_dlc-eye_*'))
assert len(eye_dir) == 1

eye_h5_fn = eye_dir[0] / 'ellipses_processed.h5'
# eye_h5_fn = eye_dir[0] / 'ellipses.h5'
with h5py.File(eye_h5_fn, 'r') as h:
    print(h.keys())
    print(h['pupil'].keys())
    print(h['eye'].keys())
    print(h['pupil']['axis0'][:])

<KeysViewHDF5 ['cr', 'eye', 'pupil']>
<KeysViewHDF5 ['axis0', 'axis1', 'block0_items', 'block0_values', 'block1_items', 'block1_values']>
<KeysViewHDF5 ['axis0', 'axis1', 'block0_items', 'block0_values', 'block1_items', 'block1_values']>
[b'center_x' b'center_y' b'width' b'height' b'phi'
 b'pupil_average_confidence' b'pupil_area' b'pupil_is_bad_frame']


In [27]:
eye_raw_fn = list(eye_dir[0].glob('*resnet50_*.h5'))[0]
with h5py.File(eye_raw_fn, 'r') as h:
    print(h.keys())
    print(h['df_with_missing'].keys())
    print(h['df_with_missing']['table'][:])

<KeysViewHDF5 ['df_with_missing']>
<KeysViewHDF5 ['_i_table', 'table']>
[(     0, [ 6.33884335e+00,  7.72512207e+01,  1.82653416e-03,  6.24676666e+01,  1.99652130e+02,  3.65068251e-03,  5.97942200e+01,  2.05098557e+02,  5.09457104e-03,  6.19379807e+01,  2.00198074e+02,  2.04754993e-02,  6.09929810e+01,  1.97884903e+02,  1.46581922e-02,  6.47720032e+01,  1.73709076e+02,  1.47243375e-02,  6.33734016e+01,  1.65902313e+02,  1.24610318e-02,  5.77698784e+01,  1.92075073e+02,  6.78498857e-03,  5.87980461e+01,  1.92362122e+02,  1.86867197e-03,  3.26299095e+00,  1.55936676e+02,  1.23810431e-03,  5.22358751e+00,  7.45234756e+01,  7.52468535e-04,  6.16038208e+01,  1.96104324e+02,  1.27416616e-03, -3.07590961e-01,  1.53588181e+02,  4.12408775e-03,  4.73304987e+00,  1.56218582e+02,  1.98574411e-03,  3.57965565e+00,  3.33953953e+00,  2.16960185e-03,  9.56604004e+01,  2.04954910e+02,  5.03960298e-03,  4.57403564e+00,  8.73029938e+01,  2.37481464e-02,  4.22407799e+01,  2.02470993e+02,  6.65119290e-03,

In [38]:
def load_eye_tracking_hdf(eye_tracking_file: Path) -> pd.DataFrame:
    """Load a DeepLabCut hdf5 file containing eye tracking data into a
    dataframe.

    Note: The eye tracking hdf5 file contains 3 separate dataframes. One for
    corneal reflection (cr), eye, and pupil ellipse fits. This function
    loads and returns this data as a single dataframe.

    Parameters
    ----------
    eye_tracking_file : Path
        Path to an hdf5 file produced by the DeepLabCut eye tracking pipeline.
        The hdf5 file will contain the following keys: "cr", "eye", "pupil".
        Each key has an associated dataframe with the following
        columns: "center_x", "center_y", "height", "width", "phi".

    Returns
    -------
    pd.DataFrame
        A dataframe containing combined corneal reflection (cr), eyelid (eye),
        and pupil data. Column names for each field will be renamed by
        prepending the field name. (e.g. center_x -> eye_center_x)
    """
    eye_tracking_fields = ["cr", "eye", "pupil"]

    eye_tracking_dfs = []
    for field_name in eye_tracking_fields:
        field_data = pd.read_hdf(eye_tracking_file, key=field_name)
        new_col_name_map = {col_name: f"{field_name}_{col_name}"
                            for col_name in field_data.columns}
        field_data.rename(new_col_name_map, axis=1, inplace=True)
        eye_tracking_dfs.append(field_data)

    eye_tracking_data = pd.concat(eye_tracking_dfs, axis=1)
    eye_tracking_data.index.name = 'frame'

    # Values in the hdf5 may be complex (likely an artifact of the ellipse
    # fitting process). Take only the real component.
    eye_tracking_data = eye_tracking_data.apply(lambda x: np.real(x.to_numpy()))  # noqa: E501

    return eye_tracking_data.astype(float)

In [40]:
eye_tracking_data = load_eye_tracking_hdf(eye_h5_fn)

[autoreload of _frozen_importlib failed: Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/opt/conda/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
  File "/opt/conda/lib/python3.9/importlib/__init__.py", line 166, in reload
    spec = module.__spec__ = _bootstrap._find_spec(name, pkgpath, target)
AttributeError: module 'importlib._bootstrap' has no attribute '_find_spec'
]
[autoreload of _frozen_importlib_external failed: Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/opt/conda/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
  File "/opt/conda/lib/python3.9/importlib/__ini

TypeError: descriptor '_str' for 'PurePath' objects doesn't apply to a 'PosixPath' object

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 725, in start
    self.io_loop.start()
  File "/opt/conda/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
    self.asyncio_loop.run_forever()
  File "/opt/conda/lib/python3.9/asyncio/base_events.py", line 601, in run_forever
    self._run_once()
  File "/opt/conda/lib/python3.9/asyncio/base_events.py", line 1891, in _run_once
    if handle._cancelled:
TypeError: descriptor '_cancelled' for

: 

: 